## Fandango Movie Ratings

![Image](https://www.counton2.com/wp-content/uploads/sites/7/2018/07/movie-popcorn-entertainment_1530120399830_382133_ver1.0_47007319_ver1.0.jpg?w=1280)

### Introduction

#### Context
In october 15, 2015 a data jourmalist Walt Hickey published his [article](https://fivethirtyeight.com/features/fandango-movies-ratings/) about [Fandango's rating system](https://www.fandango.com/). He analyzed movie ratings data and found strong evidence to suggest that Fandango's rating system was biased:
* The actual rating was almost always rounded up to the nearest half-star
* In the case of 8% of the ratings analyzed, the rounding up was done to the nearest whole star
* For one movie rating, the rounding off was completely bizarre: from a rating of 4 in the HTML of the page to a displayed rating of 5 stars

After that Fandango's officials replied that it was a bug in their system, and they promised to fix it as soon as possible. Presumably, this has already happened, but we can't tell for sure since the actual rating value isn't displayed anymore.

In this project we'll analyze more recent movie ratings data to determine whether there has been any change in Fandango's rating system after Hickey's article. We'll compare the system's characteristics previous and after the analysis for that purpose. And we have ready-made data for it:

* The data Walt Hickey collected to analyze for his article

* Movie ratings data for movies released in 2016 and 2017

Both data sets can be downloaded from this [Github repository](https://github.com/mircealex/Movie_ratings_2016_17).

#### First look

Let's read both files and take a look in their structure.

In [1]:
import pandas as pd

hickey = pd.read_csv('fandango_score_comparison.csv')
fresh = pd.read_csv('movie_ratings_16_17.csv')

hickey.head()

,FILM,RottenTomatoes,RottenTomatoes_User,Metacritic,Metacritic_User,IMDB,Fandango_Stars,Fandango_Ratingvalue,RT_norm,RT_user_norm,...,IMDB_norm,RT_norm_round,RT_user_norm_round,Metacritic_norm_round,Metacritic_user_norm_round,IMDB_norm_round,Metacritic_user_vote_count,IMDB_user_vote_count,Fandango_votes,Fandango_Difference
0,Avengers: Age of Ultron (2015),74,86,66,7.1,7.8,5.0,4.5,3.70,4.3,...,3.90,3.5,4.5,3.5,3.5,4.0,1330,271107,14846,0.5
1,Cinderella (2015),85,80,67,7.5,7.1,5.0,4.5,4.25,4.0,...,3.55,4.5,4.0,3.5,4.0,3.5,249,65709,12640,0.5
2,Ant-Man (2015),80,90,64,8.1,7.8,5.0,4.5,4.00,4.5,...,3.90,4.0,4.5,3.0,4.0,4.0,627,103660,12055,0.5
3,Do You Believe? (2015),18,84,22,4.7,5.4,5.0,4.5,0.90,4.2,...,2.70,1.0,4.0,1.0,2.5,2.5,31,3136,1793,0.5
4,Hot Tub Time Machine 2 (2015),14,28,29,3.4,5.1,3.5,3.0,0.70,1.4,...,2.55,0.5,1.5,1.5,1.5,2.5,88,19560,1021,0.5


In [2]:
fresh.head()

,movie,year,metascore,imdb,tmeter,audience,fandango,n_metascore,n_imdb,n_tmeter,n_audience,nr_metascore,nr_imdb,nr_tmeter,nr_audience
0,10 Cloverfield Lane,2016,76,7.2,90,79,3.5,3.80,3.60,4.50,3.95,4.0,3.5,4.5,4.0
1,13 Hours,2016,48,7.3,50,83,4.5,2.40,3.65,2.50,4.15,2.5,3.5,2.5,4.0
2,A Cure for Wellness,2016,47,6.6,40,47,3.0,2.35,3.30,2.00,2.35,2.5,3.5,2.0,2.5
3,A Dog's Purpose,2017,43,5.2,33,76,4.5,2.15,2.60,1.65,3.80,2.0,2.5,1.5,4.0
4,A Hologram for the King,2016,58,6.1,70,57,3.0,2.90,3.05,3.50,2.85,3.0,3.0,3.5,3.0


### Analysis

#### A bit of cleaning

There are a lot of unnecessary columns. We'll separate only usefull:
* `FILM`, `Fandango_Stars`, `Fandango_Ratingvalue`, `Fandango_votes` and `Fandango_Difference` for the Hickey's set
* `movie`, `year` and `fandango` for the 2016-2017 set

In [3]:
before_article = hickey[['FILM', 'Fandango_Stars', 'Fandango_Ratingvalue',
                         'Fandango_votes', 'Fandango_Difference']].copy()

after_article = fresh[['movie', 'year', 'fandango']].copy()

before_article.head()

,FILM,Fandango_Stars,Fandango_Ratingvalue,Fandango_votes,Fandango_Difference
0,Avengers: Age of Ultron (2015),5.0,4.5,14846,0.5
1,Cinderella (2015),5.0,4.5,12640,0.5
2,Ant-Man (2015),5.0,4.5,12055,0.5
3,Do You Believe? (2015),5.0,4.5,1793,0.5
4,Hot Tub Time Machine 2 (2015),3.5,3.0,1021,0.5


In [4]:
after_article.head()

,movie,year,fandango
0,10 Cloverfield Lane,2016,3.5
1,13 Hours,2016,4.5
2,A Cure for Wellness,2016,3.0
3,A Dog's Purpose,2017,4.5
4,A Hologram for the King,2016,3.0


Much better! Now let's recall our purpose. We must determine whether there has been any change in Fandango's rating system or not. So population of interest for our goal - all the movies that Fandango's system has.

But we'll work with two samples. To spread results to the whole population these **samples must be representative**. Fortunately to find it out we have `README.md` files that describe how samples were gathered.

The [Hickey's file](https://github.com/fivethirtyeight/data/blob/master/fandango/README.md) tells us that the data set contains films that have:
* a Rotten Tomatoes rating, a RT User rating, a Metacritic score, a Metacritic User score, and IMDb score
* at least 30 fan reviews on Fandango
* tickets on sale in 2015

And the [2016-1207 file](https://github.com/mircealex/Movie_ratings_2016_17/blob/master/README.md):
* the most popular movies (with a significant number of votes (no threshold))
* released in 2016 and 2017

It's clearly that sampling in both cases was **selective, not random**. Data was gathered specifically for analyst's purposes. Given this, it cannot be said that the samples are representative. Most likely they are not.


#### Tweaking the goal

At this point, we have at least two alternatives: either we collect new data, either we change the goal of our analysis by placing some limitations on it. The second one seems a much faster choice. Also, it's quasi-impossible to collect a new sample previous to Hickey's analysis at this moment in time.

So instead of finding any change in Fandango's rating system after Hickey's article we'll focus on **ratings changes  for popular movies between 2015 and 2016**.

#### Isolating the samples

With the new goal, we now have two populations that we want to describe and compare with each other:
* all Fandango's ratings for popular movies released in **2015**
* all Fandango's ratings for popular movies released in **2016**

To define term "popular" we'll use Hickey's benchmark of 30 fan ratings. Let's check it.

In [5]:
(before_article['Fandango_votes'] >= 30).all()

True

As was expected everything's fine with the Hickey's data.

There is no such info in the 2016-2017 data. Even `README.md` says the set contains only popular movies we still should check it. One quick way to do this is to sample randomly 10 movies from whole data set and find number of fan ratings manually.

We want to find that at least **8** of them satisfy the condition.

In [6]:
sample_10 = after_article.sample(10, random_state=0)
sample_10

,movie,year,fandango
197,The Take (Bastille Day),2016,4.0
37,Come and Find Me,2016,4.0
89,Kickboxer,2016,4.0
176,The Founder,2016,4.0
170,The Darkness,2016,2.5
75,Ice Age: Collision Course,2016,4.0
96,Lion,2016,4.0
137,Ride Along 2,2016,4.0
5,A Monster Calls,2016,4.0
83,Jane Got a Gun,2016,3.5


In [7]:
%%html
#Left align tables
<style> table {margin-left: 0 !important;} </style>

It turned out that Fandango's website uses [Rotten tomatoes](https://www.rottentomatoes.com/) ratings. So looking up these movies at the time of writing (October 2021) we have this:

| Movie                      | Votes         |
|----------------------------|---------------|
| The Take (Bastille Day)    | 2500+         |
| Come and Find Me           | 500+          |
| Kickboxer                  | 1000+         |
| The Founder                | 25,000+       |
| The Darkness               | 5,000+        |
| Ice Age: Collision Course  | 25,000+       |
| Lion                       | 25,000+       |
| Ride Along 2	             | 25,000+       |
| A Monster Calls            | 25,000+       |
| Jane Got a Gun             | 10,000+       |

Every movie from for our small sample has far more than 30 ratings. It reflects overall tendency in whole data set and we can be more confident about it's representativeness.

Also we need isolate movies by years: keep only 2015 in the `before_article` and only 2016 in the `after_article`. Before that we have to extract years from the `FILM` column in the first data set.

In [18]:
before_article['year'] = before_article['FILM'].str.extract(r'(?<=\()(\d{4})(?=\))')

before_2015 = before_article[before_article['year'] == '2015'].copy()
after_2016 = after_article[after_article['year'] == 2016].copy()

print('''2015 - {} movies
2016 - {} movies'''.format(len(before_2015), len(after_2016)))

2015 - 129 movies
2016 - 191 movies


After all these preliminary steps, we can now start analyzing the two samples we isolated before. Once again, our goal is to determine whether there's any difference between Fandango's ratings for popular movies in 2015 and Fandango's ratings for popular movies in 2016